In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


## Objectif
- prédire le temps de retard moyen des trains sur une ligne sur un mois
--> predire retard_moyen_arrivee? ou retard_moyen_tous_trains_arrivee?
- prédire la cause du retard (en pourcentage pour chaque cause potentielle) sur une ligne sur un mois

- alternative --> fusionner toutes les lignes sur un mois et predire sur un mois pour toute les lignes à la fois

## Questions

### faire deux modèles distincts ou dépendants pour ces deux problématique?
- si dépendant --> NN avec une regression et des classifs en sortie
- si indépendant --> on a le choix
- mix --> on prédit la regression d'abord puis la classif en se servant des resultats de la regression ( ou inversement)

### Modèles independants pour chaque ligne de train ou tout lier?
- si indépendant, plus facile à gérer on peut facilement créer des séries temporelles, mais beaucoup(beaucoup) de modèle à créer
- si liée, on peut exploiter des correlations en plus et on n'a qu'un modèle, mais on doit encoder l'année, le mois et toutes les gares (avec un one hot ça va faire beaucoup de colonne, avec un ordinal on introduit une relation d'ordre qui n'existe pas, remarque on aurait pas ce soucis avec des arbres de décision)

### Analyse à faire 
- autocorrelation mois et année sur le temps de retard et les causes du retard
- regarder correlation retard moyen entre paris-lyon et lyon-paris (pour tout les couples de gares)
- pleins d'autres trucs qui nous permettront d'enlever/modifier/rajouter des colonnes

### Données qu'on peut rajouter
- longueur de chaque lignes (pas forcement nécessaire?)
- nombre d'habitants dans chaques ville

### Infos importantes
- https://www.data.gouv.fr/fr/datasets/regularite-mensuelle-tgv-par-liaisons/

### Entree/Sortie
- entrées: date (mois), service,gare départ/arrivée

In [2]:
df = pd.read_csv("regularite-mensuelle-tgv-aqst.csv",encoding="latin-1")


In [10]:
df.columns

Index(['date', 'service', 'gare_depart', 'gare_arrivee', 'duree_moyenne',
       'nb_train_prevu', 'nb_annulation', 'commentaire_annulation',
       'nb_train_depart_retard', 'retard_moyen_depart',
       'retard_moyen_tous_trains_depart', 'commentaire_retards_depart',
       'nb_train_retard_arrivee', 'retard_moyen_arrivee',
       'retard_moyen_tous_trains_arrivee', 'commentaires_retard_arrivee',
       'nb_train_retard_sup_15', 'retard_moyen_trains_retard_sup15',
       'nb_train_retard_sup_30', 'nb_train_retard_sup_60',
       'prct_cause_externe', 'prct_cause_infra', 'prct_cause_gestion_trafic',
       'prct_cause_materiel_roulant', 'prct_cause_gestion_gare',
       'prct_cause_prise_en_charge_voyageurs'],
      dtype='object')

In [62]:
print(len(set(df["gare_depart"].tolist()))) #pd.get_dummies
print(len(set(df["gare_arrivee"].tolist()))) #pd.get_dummies
print(len(df["date"].tolist())) 

59
59
8154


In [74]:
def clean_df(df):
    # Add commands to clean dataframe
    df = df.dropna(axis=1, how='all')
    df = df.drop("commentaires_retard_arrivee", axis=1) # str and not relevant for model
    return df

clean_df = clean_df(df) #Test
# Dataset cleaning should be done on train and test set separately -> ensure reproducibility


In [75]:
print(len(clean_df.columns))
print(len(df.columns))

23
26


In [55]:
def get_train_test_set(df):
    train_idx = []
    test_idx = []
    for i, d in enumerate(df["date"].tolist()):
        month, day, year = d.split('/')
        if int(year)<2023:
            train_idx.append(i)
        else:
            test_idx.append(i)
    
    train_set = df.iloc[train_idx].copy(deep=True)
    test_set = df.iloc[test_idx].copy(deep=True)
    
    return train_set, test_set

train_set, test_set = get_train_test_set(df)


split = [(len(train_idx)/len(df))*100, (len(test_idx)/len(df))*100]
print(f"Lenghts of train/test: {len(train_idx)}/{len(test_idx)}")
print(f"split train/test: {round(split[0],2)}/{round(split[1],2)}")

Lenghts of train/test: 7428/726
split train/test: 91.1/8.9


In [53]:
set(train_set["service"].tolist())

{'International', 'National'}

test with date transform to year, months over years, days